# Direct inference

In [ ]:
# example of inference for direct submission
!cd GWC_YOLO && python detect.py --img-size 800 --name fold{index} --weights ../{name}/fold{index}.pt --source ../test --augment --nosave --save-csv --conf-thres 0.50

# Ensemble using weighted boxes fusion

Put the folders containing labels txt files in the current directory. Each txt file contains *cls, xywh, conf*.

In [ ]:
# example of inference for ensemble
!cd GWC_YOLO && python detect.py --img-size 800 --name fold{index} --weights ../{name}/fold{index}.pt --source ../test --augment --nosave --save-txt --save-conf --conf-thres 0.50
!mv GWC_YOLO/runs/detect/fold{index}/labels fold{index}

In [ ]:
import os
import pandas as pd

# Put the names of the folders here
folders = ['fold1', 'fold3', 'fold3']
# Put the weights attached to each folder here
weights_list = [1, 1, 1]
# Put the indexes of the weights whose confidences should be *sqrt*ed here
# This is especially useful when ensembling models predicted using *master* branch and *original* branch
sqrt_indexes = [2]
files = [os.listdir(folder) for folder in folders]
submission = pd.read_csv('submission.csv')

In [ ]:
!pip install aicrowd-cli
!pip install ensemble_boxes

In [ ]:
import numpy as np

def parse_box(box):
  x, y, w, h = box
  return max(0, x - w / 2), max(0, y - h / 2), min(1, x + w / 2), min(1, y + h / 2)

def normalize(boxes):
  return [[entry / 1024 for entry in box] for box in boxes]

def de_normalize(boxes):
  return [[entry * 1024 for entry in box] for box in boxes]

def parse_file(file_name, current):
  boxes = []
  scores = []
  labels = []
  with open(file_name) as f:
    for line in f:
      cls, *xywh, conf = list(map(float, line.split()))
      boxes.append(parse_box(xywh))
      if current in sqrt_indexes:
        scores.append(np.sqrt(conf))
      else:
        scores.append(conf)
      labels.append(0)
  return boxes, scores, labels

def encodeBoxes(boxes, scores, score_thr=0.0):
  strboxes = ";".join([" ".join([str(round(i)) for i in box]) for (box, score) in zip(boxes, scores) if score > score_thr])

  if len(strboxes) == 0:
    strboxes = "no_box"
  
  return strboxes

In [ ]:
from ensemble_boxes import weighted_boxes_fusion
from tqdm import tqdm
import os, shutil


for index, line in tqdm(enumerate(submission.iterrows())):
  img_name = line[1]['image_name']
  txt_name = img_name + '.txt'
  boxes_list = [[] for _ in range(len(folders))]
  scores_list = [[] for _ in range(len(folders))]
  labels_list = [[] for _ in range(len(folders))]
  for current, folder in enumerate(folders):
    # assert txt_name in files[current]
    boxes, scores, labels = parse_file(os.path.join(folder, txt_name), current)
    boxes_list[current] = boxes
    scores_list[current] = scores
    labels_list[current] = labels
  final_boxes, final_scores, _ = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights_list)

  submission.at[index, 'PredString'] = encodeBoxes(de_normalize(final_boxes), final_scores)

submission.head()

In [ ]:
submission.to_csv('ensemble.csv')

In [ ]:
# For sanity check
!cp drive/MyDrive/GWC_data/test.zip ./
!unzip test.zip

import pandas as pd
ensemble = pd.read_csv('ensemble.csv')
image_name = ensemble['image_name'].tolist()[0]
boxes = ensemble['PredString'].tolist()[0]

import matplotlib.pyplot as plt
import cv2
%matplotlib inline

def visualize(image_path, boxes):
  boxes = [box.split(' ') for box in boxes.split(';')]
  image = cv2.imread(image_path)
  for (x1, y1, x2, y2) in boxes:
    cv2.rectangle(image,(int(x1),int(y1)),(int(x2),int(y2)),(255,0,0),5)
  plt.rcParams["figure.figsize"] = (10, 10)
  plt.imshow(image[...,::-1])

visualize(f'test/{image_name}.png', boxes)

## Making Direct Submission thought Aicrowd CLI

In [ ]:
### Please enter your API Key from [here](https://www.aicrowd.com/participants/me).
API_KEY = "api_key" 
!aicrowd login --api-key $API_KEY
!aicrowd submission create -c global-wheat-challenge-2021 -f ensemble.csv